- How to tune multiple parameters
- How to perform random search hyper parameter tuning
- Understand diff b/w GridSearchCV and RandomizedSearchCV

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_validate
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [2]:
data = pd.read_csv('insurance.csv')
data.head()

age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520

In [3]:
ohe = pd.get_dummies(data.drop(columns=['age','bmi','children','charges']), drop_first=True) 

x = data.drop(columns='charges')
y = data['charges']

ohe_x = pd.concat([x, ohe], axis=1)
ohe_x = ohe_x.drop(columns=['sex','smoker','region'])
ohe_x.head()

age     bmi  children  sex_male  smoker_yes  region_northwest  \
0   19  27.900         0         0           1                 0   
1   18  33.770         1         1           0                 0   
2   28  33.000         3         1           0                 0   
3   33  22.705         0         1           0                 1   
4   32  28.880         0         1           0                 1   

   region_southeast  region_southwest  
0                 0                 1  
1                 1                 0  
2                 1                 0  
3                 0                 0  
4                 0                 0

In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(ohe_x, y, test_size = .2, random_state=3)
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((1070, 8), (1070,), (268, 8), (268,))

## Tuning multiple hyper parameters
* Performing cross multiplication

In [5]:
params = {'max_depth':list(range(2,7)), 'min_samples_split':[5,10,15]}

for x in params['max_depth']:
    for y in params['min_samples_split']:
        print(x, y)
    print('-'*100)

2 5
2 10
2 15
----------------------------------------------------------------------------------------------------
3 5
3 10
3 15
----------------------------------------------------------------------------------------------------
4 5
4 10
4 15
----------------------------------------------------------------------------------------------------
5 5
5 10
5 15
----------------------------------------------------------------------------------------------------
6 5
6 10
6 15
----------------------------------------------------------------------------------------------------


# Perform a grid search

In [6]:
dtr = DecisionTreeRegressor()

gs = GridSearchCV(estimator=dtr, param_grid=params, scoring='r2', cv=5).fit(xtrain, ytrain)
##### Manual
gs.best_params_, gs.best_score_

({'max_depth': 4, 'min_samples_split': 5}, 0.8442256526379934)

* Problem with GridSearchCV is that the number of combinations will increase with multiple hyperparameters like above
* With multiple hyperparameters through GridSearchCV, whole process will take lot of time with large data 
* Due to such probs we use RandomizedSearchCV

**RandomizedSearchCV** : Instead of making combinations with all the hyperparameters, it will select few of the combinations and does hyperparameter tuning.
* Due to which no of combinations are less and will yeild optimal results faster

In [7]:
params1 = {'max_depth':list(range(2,50)), 'min_samples_split':[5,10,15,20,25,30]}

dtr = DecisionTreeRegressor()

rs = RandomizedSearchCV(estimator=dtr, param_distributions=params1, scoring='r2', cv=5).fit(xtrain, ytrain)

#gs = GridSearchCV(estimator=dtr, param_grid=params, scoring='r2', cv=5).fit(xtrain, ytrain)


##### Manual
rs.best_params_, rs.best_score_

({'min_samples_split': 25, 'max_depth': 5}, 0.8446653273789355)

- Above result will be close to original optimal values due to randomized search
- But sometimes results won't be close to original optimal values

In [9]:
# If we want to know results of R2 scores for other max_depths use -> cv_results_
df_cv_results = pd.DataFrame(rs.cv_results_)
df_cv_results

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0       0.003101      0.000834         0.001854        0.000795   
1       0.004237      0.000916         0.002727        0.000856   
2       0.003176      0.000658         0.002070        0.000786   
3       0.003128      0.001147         0.001629        0.000736   
4       0.003863      0.001461         0.002019        0.000245   
5       0.003013      0.000870         0.001321        0.000500   
6       0.003192      0.000767         0.001435        0.000357   
7       0.002758      0.000800         0.001671        0.000407   
8       0.002604      0.000337         0.001015        0.000083   
9       0.002386      0.000161         0.001421        0.000333   

  param_min_samples_split param_max_depth  \
0                      30               6   
1                       5              47   
2                      15              20   
3                      20              15   
4                      20              34   
5                      20              46   
6                      20              20   
7                      25               5   
8                      15              11   
9                      20               7   

                                       params  split0_test_score  \
0   {'min_samples_split': 30, 'max_depth': 6}           0.874720   
1   {'min_samples_split': 5, 'max_depth': 47}           0.739254   
2  {'min_samples_split': 15, 'max_depth': 20}           0.836564   
3  {'min_samples_split': 20, 'max_depth': 15}           0.838528   
4  {'min_samples_split': 20, 'max_depth': 34}           0.838528   
5  {'min_samples_split': 20, 'max_depth': 46}           0.838528   
6  {'min_samples_split': 20, 'max_depth': 20}           0.838528   
7   {'min_samples_split': 25, 'max_depth': 5}           0.878554   
8  {'min_samples_split': 15, 'max_depth': 11}           0.838288   
9   {'min_samples_split': 20, 'max_depth': 7}           0.856050   

   split1_test_score  split2_test_score  split3_test_score  split4_test_score  \
0           0.847394           0.818756           0.850710           0.801701   
1           0.762437           0.728506           0.769859           0.710939   
2           0.824082           0.774789           0.797103           0.773329   
3           0.833769           0.785449           0.811917           0.785415   
4           0.833769           0.785449           0.811917           0.785415   
5           0.833769           0.785449           0.811917           0.785415   
6           0.833769           0.785449           0.811917           0.785415   
7           0.853756           0.826455           0.850340           0.814222   
8           0.823946           0.776141           0.798767           0.777272   
9           0.846983           0.802616           0.823181           0.793957   

   mean_test_score  std_test_score  rank_test_score  
0         0.838656        0.025628                2  
1         0.742199        0.021670               10  
2         0.801173        0.025555                9  
3         0.811016        0.022735                4  
4         0.811016        0.022735                4  
5         0.811016        0.022735                4  
6         0.811016        0.022735                4  
7         0.844665        0.022457                1  
8         0.802883        0.024840                8  
9         0.824558        0.024143                3

In [10]:
df_cv_results[['params']]

params
0   {'min_samples_split': 30, 'max_depth': 6}
1   {'min_samples_split': 5, 'max_depth': 47}
2  {'min_samples_split': 15, 'max_depth': 20}
3  {'min_samples_split': 20, 'max_depth': 15}
4  {'min_samples_split': 20, 'max_depth': 34}
5  {'min_samples_split': 20, 'max_depth': 46}
6  {'min_samples_split': 20, 'max_depth': 20}
7   {'min_samples_split': 25, 'max_depth': 5}
8  {'min_samples_split': 15, 'max_depth': 11}
9   {'min_samples_split': 20, 'max_depth': 7}

# * Note :- 1st try out RandomizedSearchCV to know the vicinity of combinations and then try out GridSearchCV to know the actual optimal values 